# Plasmodium data

This notebook was designed to read and analyze the `.dv` *(deltavision)* files. In our case these files contain time-series frames of plasmodium.

### What does this notebook do?  
1. First, it traverses through all directories and subdirectories in the current directory and finds all the `.dv` files.
   - A list of the file paths is stored in the variable `dv_files`.
   - A dropdown menu is created from the list of files and is then shown to the user.
2. User can select the desired file via a dropdown menu.
3. After a file is selected, a 2-row by 4-column plot shows up. The plot contains:
   1. First column: the captured frame of plasmodium at time `t`.
   2. The rest of the columns: the line plot of the **minimum, average, and maximum pixel intensity** of the frame at time `t` in the same order.

### Some notes about the data
1. The data was huge and I had to exclude them from the `data` directory. But you can and should add your directories containing the `.dv` files under the `data` directory.
2. Upon examining the header of the files I realized that there are **two channels** of **41 frames** in each `.dv` file. Hence the 2-row-plot. Each row of the plot corresponds to a channel in the `.dv. file.
3. The `.dv` file has a header containing a great deal of useful information. The enum class `headerTuples` was created to make it easier and more human readabla to extract the desired header value. I.e. instead of 

In [1]:
from mrc import DVFile, imread
import numpy as np
from matplotlib import pyplot as plt
import imageio
import cv2
from PIL import Image, ImageStat
import os
from enum import Enum
import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import display, clear_output
import warnings
from ipywidgets import interact, interact_manual, widgets, Dropdown
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
from PIL import Image
%matplotlib inline


In [2]:
dv_files = []


class headerTuples(Enum):
    photosensorReading = 0
    timeStampSeconds = 1
    stageXCoord = 2
    stageYCoord = 3
    stageZCoord = 4
    minInten = 5
    maxInten = 6
    meanInten = 7
    expTime = 8
    ndFilter = 9
    exWavelen = 10
    emWavelen = 11
    intenScaling = 12
    energyConvFactor = 13


def read_as_nparray(filename: str):
    img_array = imread(filename)
    assert len(
        img_array) > 0, f"file {filename} is not a valid DV file or is corrupted."
    return img_array


def save_nparray_image_as_png(nparray_image: np.ndarray, directory: str, filename: str):
    assert len(
        nparray_image.shape) == 2, "pass one image at a time. e.g. a 500 by 500 array"
    imageio.imwrite(f"{filename}.png", nparray_image)


def find_dv_files(parent_dir='.'):
    global dv_files
    dv_files = []
    for root, dirs, files in os.walk(".", topdown=False):
        for name in files:
            _, ext = os.path.splitext(name)
            if ext == '.dv':
                # print(os.path.join(root, name))
                dv_files.append(os.path.join(root, name))


find_dv_files()


def load_dv_file(file_name):
    return DVFile(file_name)


def get_intensities(dv_img, type: str = 'mean'):
    intensities = []
    for i in range(81):
        if type == 'mean':
            intensities.append(dv_img.ext_hdr.frame(i)[
                headerTuples.meanInten.value])
        if type == 'min':
            intensities.append(dv_img.ext_hdr.frame(i)[
                headerTuples.minInten.value])
        if type == 'max':
            intensities.append(dv_img.ext_hdr.frame(i)[
                headerTuples.maxInten.value])
    return intensities


def get_dv_data(img: DVFile):
    return img.asarray()


In [3]:
def draw():

    @interact
    def brows_files(filename=widgets.Dropdown(options=dv_files, description="select file"), i=widgets.IntSlider(min=0, max=40, step=1, value=0)):
        dv_file = load_dv_file(filename)
        image = get_dv_data(dv_file)
        n = image.shape[1] - 1 if image is not None else 40
        all_frames = np.ndarray((82, 960, 960))
        all_frames = np.concatenate((image[0], image[1]))
        avgIntensities = []
        minIntensities = []
        maxIntensities = []
        avgIntensities = get_intensities(dv_file, 'mean')
        minIntensities = get_intensities(dv_file, 'min')
        maxIntensities = get_intensities(dv_file, 'max')

        # def mean(i, j): return np.mean(image[i, j])

        fig, axs = plt.subplots(2, 4, figsize=(20, 10))

        calcMeanIntens = []
        for j in range(len(all_frames)):
            calcMeanIntens.append(np.mean(all_frames[j]))

        calcMinIntens = []
        for j in range(len(all_frames)):
            calcMinIntens.append(np.min(all_frames[j]))

        calcMaxIntens = []
        for j in range(len(all_frames)):
            calcMaxIntens.append(np.max(all_frames[j]))

        axs[0, 0].imshow(image[0, i], cmap=plt.cm.gray_r,
                         interpolation='nearest')
        axs[1, 0].imshow(image[1, i], cmap=plt.cm.gray_r,
                         interpolation='nearest')

        axs[0, 1].plot(minIntensities[:n])
        axs[0, 1].plot(calcMinIntens[:n])
        axs[0, 1].plot(i, minIntensities[i], 'go')
        axs[1, 1].plot(minIntensities[n+1:])
        axs[1, 1].plot(calcMinIntens[n+1:])
        axs[1, 1].plot(i, minIntensities[n+1+i], 'go')
        axs[0, 1].set_title("minimum pixel intensity")

        axs[0, 2].plot(avgIntensities[:n])
        axs[0, 2].plot(calcMeanIntens[:n])
        axs[0, 2].plot(i, avgIntensities[i], 'go')
        axs[1, 2].plot(avgIntensities[n+1:])
        axs[1, 2].plot(calcMeanIntens[n+1:])
        axs[1, 2].plot(i, avgIntensities[n+1+i], 'go')

        axs[0, 2].set_title("average pixel intensity")

        axs[0, 3].plot(maxIntensities[:n])
        axs[0, 3].plot(calcMaxIntens[:n])
        axs[0, 3].plot(i, maxIntensities[i], 'go')
        axs[1, 3].plot(maxIntensities[n+1:])
        axs[1, 3].plot(calcMaxIntens[n+1:])
        axs[1, 3].plot(i, maxIntensities[n+1+i], 'go')
        axs[0, 3].set_title("maximum pixel intensity")

        fig.tight_layout()


draw()


interactive(children=(Dropdown(description='select file', options=(), value=None), IntSlider(value=0, descript…